In [86]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    return cls


class Account:
    pass


@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass


@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [87]:
b1_savings = Bank1Savings()
b2_checking = Bank2Checking()

print(b1_savings.account_type)
print(b2_checking.account_type)

savings
checking


Enhance the `Account` class with another decorator

In [88]:
def account_type(acc_type):
    def inner(cls):
        cls.account_type = acc_type
        return cls
    return inner

In [89]:
@account_type('checking')
class BankChecking(Account):
    pass

In [90]:
bc = BankChecking()
print(bc.account_type)

checking


Class decorator for all callable methods

In [104]:
def func_logger(cls):
    @wraps(cls)
    def inner(*args, **kwargs):
        print(f"Calling {cls.__name__} with args: {args}, kwargs: {kwargs}")
        return cls(*args, **kwargs)
    return inner

In [105]:
class Person:
    def __init__(self, name):
        self.name = name

    @func_logger
    def greet(self):
        print(f"Hello, {self.name}!")

    @func_logger
    def farewell(self):
        print(f"Goodbye, {self.name}!")


In [106]:
p =Person("John")
p.greet()

Calling greet with args: (<__main__.Person object at 0x000001E14C21F850>,), kwargs: {}
Hello, John!


In [121]:
from functools import wraps


def logger(cls):
    for attr_name, attr_value in cls.__dict__.items():
        if callable(attr_value):
            print('attr_name', attr_name)
            print('attr_type', type(attr_value))
            setattr(cls, attr_name, func_logger(attr_value))
        elif isinstance(attr_value, staticmethod):
            # Handle static methods
            func = attr_value.__func__
            setattr(cls, attr_name, staticmethod(func_logger(func)))
        elif isinstance(attr_value, classmethod):
            # Handle class methods
            func = attr_value.__func__
            setattr(cls, attr_name, classmethod(func_logger(func)))
        elif isinstance(attr_value, property):
            # Handle properties
            if attr_value.fget:
                attr_value = attr_value.getter(func_logger(attr_value.fget))
            if attr_value.fset:
                attr_value = attr_value.setter(func_logger(attr_value.fset))
            setattr(cls, attr_name, attr_value)


    return cls

In [122]:
@logger
class SomeObject:



    def __init__(self):
        print("SomeObject initialized")

    @property
    def version(self):
        return "1.0"

    @version.setter
    def version(self, value):
        print(f"Setting version to {value}")

    def method1(self, x):
        print(f"Method 1 called with {x}")

    @staticmethod
    def static_method(y):
        print(f"Static Method 2 called with {y}")

    @classmethod
    def class_method(cls, z):
        print(f"Class Method 3 called with {z}")

attr_name __init__
attr_type <class 'function'>
attr_name method1
attr_type <class 'function'>
attr_name static_method
attr_type <class 'staticmethod'>


In [125]:
s = SomeObject()
s.method1(10)
SomeObject.static_method(20)
SomeObject.class_method(30)
s.version
s.version = "2.0"



Calling __init__ with args: (<__main__.SomeObject object at 0x000001E14C21D6D0>,), kwargs: {}
SomeObject initialized
Calling method1 with args: (<__main__.SomeObject object at 0x000001E14C21D6D0>, 10), kwargs: {}
Method 1 called with 10
Calling static_method with args: (20,), kwargs: {}
Static Method 2 called with 20
Calling class_method with args: (<class '__main__.SomeObject'>, 30), kwargs: {}
Class Method 3 called with 30
Calling version with args: (<__main__.SomeObject object at 0x000001E14C21D6D0>,), kwargs: {}
Calling version with args: (<__main__.SomeObject object at 0x000001E14C21D6D0>, '2.0'), kwargs: {}
Setting version to 2.0
